In [1]:
from path import Path
from PIL import Image
import cv2
import random
import pandas as pd
import pickle

In [2]:
def arg_parse():
    parser = argparse.ArgumentParser()
    parser = argparse.ArgumentParser(
                        prog="annotation.py", 
                        usage="annotation.py -n <<num_of_evaluation>>", 
                        description="", 
                        add_help = True 
                        )

    parser.add_argument("-n", "--num", 
                        help = "num of evaluation",
                        type = int,
                        default = None)
    
    args = parser.parse_args()
    return args


In [3]:
def get_filepath_list(dir_path):
    imgs = Path(dir_path).files('*.png')
    imgs += Path(dir_path).files('*.jpg')
    imgs += Path(dir_path).files('*.jpeg')
    
    return imgs

In [4]:
def hconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv2.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation)
                      for im in im_list]
    return cv2.hconcat(im_list_resize)

In [5]:
def evaluate_images(path_list, rand=False, n_shows=None, username=None):
    
    df = pd.DataFrame(columns=['filename', 'score', 'user'])
    filename_list = []
    score_list = []
    rep_list = [ord(str(i)) for i in range(1, 6)]
    key_q = ord('q')
    
    if rand:
        path_list = random.sample(path_list, len(path_list))
        
    if n_shows is None:
        n_shows = len(path_list)
        
    for path in path_list[:n_shows]:
        img = cv2.imread(path)
        cv2.namedWindow("image", cv2.WINDOW_KEEPRATIO | cv2.WINDOW_NORMAL)
        cv2.resizeWindow('image', 800, 600)
        cv2.imshow('image', img)
        
        key = 0
        while ((key not in rep_list) and key is not key_q):
            key = cv2.waitKey(0)
        cv2.destroyWindow('image')
        
        if key is key_q:
            break
        
        filename_list.append(path.rsplit('/')[-1])
        score_list.append(rep_list.index(key)+1)
    
    df = pd.DataFrame()
    df['filename'] = filename_list
    df['score'] = score_list
    df['user'] = username
    
    return df

In [6]:
def evaluate_images_relative(path_list, combination_list, username=None):
    df = pd.DataFrame(columns=['filename', 'score', 'user'])
    filename_list = [path.rsplit('/')[-1] for path in path_list]
    score_list = [0 for i in range(len(path_list))]
    num_evals = [0 for i in range(len(path_list))]
    
    key_f, key_j, key_q = ord('f'), ord('j'), ord('q')
    rep_list = [key_f, key_j, key_q]
    end_flag = False
    
    for i, c_list in enumerate(combination_list):
        img1 = cv2.imread(path_list[i])
        for c in c_list:
            img2 = cv2.imread(path_list[c])
            merged = hconcat_resize_min([img1, img2])
            
            cv2.namedWindow("image", cv2.WINDOW_KEEPRATIO | cv2.WINDOW_NORMAL)
            cv2.resizeWindow('image', 1200, 450)
            cv2.moveWindow('image', 100, 200)
            cv2.imshow('image', merged)
            
            key = 0
            while key not in rep_list:
                key = cv2.waitKey(0)
            cv2.destroyWindow('image')
            
            if key is key_f:
                score_list[i] = score_list[i] + 1
                num_evals[i] = num_evals[i] + 1
                num_evals[c] = num_evals[c] + 1
            elif key is key_j:
                score_list[c] = score_list[c] + 1
                num_evals[i] = num_evals[i] + 1
                num_evals[c] = num_evals[c] + 1
            else:
                end_flag = True
                break
                
            combination_list[c].remove(i)
                
        if end_flag:
            break

    df = pd.DataFrame()
    df['filename'] = filename_list
    df['score'] = score_list
    df['num_of_evaluations'] = num_evals
    df['user'] = username
    
    return df

In [7]:
def evaluate_images_relative_random(path_list, random_combination_list, start_pos, num=None, username=None):
    
    df = pd.DataFrame(columns=['filename', 'score', 'user'])
    filename_list = [path.rsplit('/')[-1] for path in path_list]
    score_list = [0 for i in range(len(path_list))]
    num_evals = [0 for i in range(len(path_list))]
    
    key_f, key_j, key_q = ord('f'), ord('j'), ord('q')
    rep_list = [key_f, key_j, key_q]
    end_flag = False
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    if num is None:
        num = len(combination_list)
    
    for count, (i, j) in enumerate(random_combination_list[start_pos:num]):
        s1, s2 = random.sample([i, j], 2)
        img1 = cv2.imread(path_list[s1])
        img2 = cv2.imread(path_list[s2])
        merged = hconcat_resize_min([img1, img2])
        
        cv2.namedWindow("image", cv2.WINDOW_KEEPRATIO | cv2.WINDOW_NORMAL)
        cv2.resizeWindow('image', 1200, 450)
        cv2.moveWindow('image', 100, 200)
        text_pos = (merged.shape[1] - 250, merged.shape[0] - 50)
        cv2.putText(merged, "{}/{}".format(count+1, len(random_combination_list)), text_pos, font, 1.5, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow('image', merged)

        key = 0
        while key not in rep_list:
            key = cv2.waitKey(0)
        cv2.destroyWindow('image')

        if key is key_f:
            score_list[s1] = score_list[s1] + 1
            num_evals[s1] = num_evals[s1] + 1
            num_evals[s2] = num_evals[s2] + 1
        elif key is key_j:
            score_list[s2] = score_list[s2] + 1
            num_evals[s1] = num_evals[s1] + 1
            num_evals[s2] = num_evals[s2] + 1
        else:
            end_flag = True
            break

        if end_flag:
            break

    df = pd.DataFrame()
    df['filename'] = filename_list
    df['score'] = score_list
    df['num_of_evaluations'] = num_evals
    df['user'] = username
    
    return df

In [8]:
def save_evaluation_csv(df, username, save_path=None):
    if save_path is None:
        save_path = '../output/test/' + username + '.csv'
    df.to_csv(save_path)

In [9]:
def main():
    print('Please write your name : ', end='')
    username = input()
    filepath_list = get_filepath_list('./images')
    df_result = evaluate_images(filepath_list, rand=True, username=username)
    save_evaluation_csv(df_result, username)
    print('Thank you!')
    
def main_relative():
    print('Please write your name : ', end='')
    username = input()
    filepath_list = get_filepath_list('./images/omelette_rice/')[:50]
    with open('./pickle/combination_list.pickle', 'rb') as f:
        combination_list = pickle.load(f)
    df_result = evaluate_images_relative(filepath_list, combination_list, username=username)
    save_evaluation_csv(df_result, username)
    print('Thank you!')
    
def main_relative_random():
    print('Please enter your name : ', end='')
    username = input()
    print('Please enter the number of evaluate : ', end='')
    num = int(input()) * 10
    filepath_list = get_filepath_list('../images/omelette_rice_500/images/')
    
    try:
        with open('../pickle/start_position.pickle', 'rb') as f:
            start_pos = pickle.load(f)
    except:
        start_pos = 0
        
    with open('../pickle/rand_combination500_list.pickle', 'rb') as f:
        random_combination_list = pickle.load(f)
        
    df_result = evaluate_images_relative_random(filepath_list, random_combination_list, start_pos, num, username=username)
    save_evaluation_csv(df_result, username)
    
    with open('../pickle/start_position.pickle', 'wb') as f:
        start_pos = (start_pos + num) % 500
        pickle.dump(start_pos, f)
        
    print('Thank you!')

In [ ]:
if __name__=='__main__':
    main_relative_random()